In [ ]:
# Every Run

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Unzip files
!mkdir tess_data
#https://smartlaboratory.org/ravdess/
!unzip 'gdrive/My Drive/FYP Code/dataverse_files.zip' -d tess_data
!rm tess_data/MANIFEST.TXT

In [ ]:
# Every Run


# Library Imports
import os
import librosa
import librosa.display
import numpy as np
import random
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import IPython.display as ipd
from IPython.display import Audio



# Constants

SAMPLE_RATE = 22050*2
FILE_COUNT = 2800
EMOTIONS = ["","Neutral","Calm","Happy","Sad","Angry","Fearful","Disgust","Surprised"]
TESS_EMOTIONS = ["","n","","h","s","a","f","d","p"]

In [ ]:
txt = "YAF_youth_sad.wav"
txtList = txt.split("_")
print(txtList[2][0:1])

print(tessSorter(txt))

In [ ]:
def tessSorter(fileName):
  fileNameList = fileName.split("_")
  temp = fileNameList[2][0:1]
  label = TESS_EMOTIONS.index(temp)
  if fileNameList[0] == "OAF": actor = 25
  else: actor = 26

  return label, actor

In [ ]:
# Create initial np array to store audio data

i = 0

X_arr = np.zeros([FILE_COUNT,110250])
labels = []
actors = []


for file in os.listdir('tess_data/'):
  X, sample_rate = librosa.load(f'{"tess_data/"}{file}', res_type='kaiser_fast',duration=2.5,sr=SAMPLE_RATE,)
  # print(folder,file,i)
  X_arr[i,:X.shape[0]] = X

  tempLabel, tempActor =tessSorter(file)

  labels.append(tempLabel)
  actors.append(tempActor)

  i += 1

labels = np.array(labels)
actors = np.array(actors)

np.save('gdrive/My Drive/FYP Code/Save Data/tess_AudioData.npy',X_arr)
np.save('gdrive/My Drive/FYP Code/Save Data/tess_Labels.npy',labels)
np.save('gdrive/My Drive/FYP Code/Save Data/tess_Actors.npy',actors)

In [ ]:
# Load audio data

tess_AudioData = np.load('gdrive/My Drive/FYP Code/Save Data/tess_AudioData.npy')
tess_Labels = np.load('gdrive/My Drive/FYP Code/Save Data/tess_Labels.npy')
tess_Actors = np.load('gdrive/My Drive/FYP Code/Save Data/tess_Actors.npy')

In [ ]:
# Test Load

idx = random.randint(0,FILE_COUNT-1)
print(idx,tess_Labels[idx],EMOTIONS[tess_Labels[idx]],tess_Actors[idx])

Audio(tess_AudioData[idx], rate = SAMPLE_RATE, autoplay = True)


# 01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised

In [ ]:
def addNoise(X):
  meanVol = np.mean(np.abs(X))
  noiseArr = np.random.normal(0,meanVol,X.shape)
  noisyX = X+noiseArr
  return noisyX

In [ ]:
# Adding Noise

X_arr = np.zeros(tess_AudioData.shape)

for i in range(tess_AudioData.shape[0]):
  X = addNoise(tess_AudioData[i])
  X_arr[i,:X.shape[0]] = X

np.save('gdrive/My Drive/FYP Code/Save Data/tess_AudioDataWN.npy',X_arr)

In [ ]:
tess_AudioDataWN = np.load('gdrive/My Drive/FYP Code/Save Data/tess_AudioDataWN.npy')

In [ ]:
idx = random.randint(0,FILE_COUNT-1)
idx = 2300
print(idx,tess_Labels[idx],EMOTIONS[tess_Labels[idx]],tess_Actors[idx])

Audio(tess_AudioDataWN[idx], rate = SAMPLE_RATE, autoplay = True)

In [ ]:
def mixAudio(fgn,bgn,fgnRatio):
  # fgn: Foreground Noise
  # bgn: Background Noise
  output = np.zeros(110250)
  output[:fgn.shape[0]] += fgnRatio/max(fgn) * fgn
  output[:bgn.shape[0]] += (1-fgnRatio)/max(bgn) * bgn

  return output


In [ ]:
temp = mixAudio(tess_AudioData[random.randint(0,FILE_COUNT-1)],tess_AudioData[random.randint(0,FILE_COUNT-1)],0.9)
Audio(temp, rate = SAMPLE_RATE, autoplay = True)

In [ ]:
# Mixed Audio

X_arr = np.zeros(tess_AudioData.shape)

for i in range(tess_AudioData.shape[0]):
  X = mixAudio(tess_AudioData[i],tess_AudioData[random.randint(0,FILE_COUNT-1)],0.9)
  X_arr[i,:X.shape[0]] = X

np.save('gdrive/My Drive/FYP Code/Save Data/tess_AudioDataMA.npy',X_arr)

In [ ]:
tess_AudioDataMA = np.load('gdrive/My Drive/FYP Code/Save Data/tess_AudioDataMA.npy')

In [ ]:
idx = random.randint(0,FILE_COUNT-1)
print(idx,tess_Labels[idx],EMOTIONS[tess_Labels[idx]],tess_Actors[idx])

Audio(tess_AudioDataMA[idx], rate = SAMPLE_RATE, autoplay = True)

In [ ]:
tess_AudioData = []

In [ ]:
# Mixed Audio and Added Noise

X_arr = np.zeros(tess_AudioDataMA.shape)

for i in range(tess_AudioDataMA.shape[0]):
  X = addNoise(tess_AudioDataMA[i])
  X_arr[i,:X.shape[0]] = X

np.save('gdrive/My Drive/FYP Code/Save Data/tess_AudioDataMAWN.npy',X_arr)

In [ ]:
tess_AudioDataMAWN = np.load('gdrive/My Drive/FYP Code/Save Data/tess_AudioDataMAWN.npy')

In [ ]:
idx = random.randint(0,FILE_COUNT-1)
print(idx,tess_Labels[idx],EMOTIONS[tess_Labels[idx]],tess_Actors[idx])

Audio(tess_AudioDataMAWH[idx], rate = SAMPLE_RATE, autoplay = True)

In [ ]:
# Generate MFCCs
def extractMFCCs(audioData):
  i = 0
  s_arr = np.zeros([audioData.shape[0],13,216])
  for line in audioData:
    X = line
    sample_rate = np.array(SAMPLE_RATE)
    mfccs = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13)
    result = np.zeros((13,216))
    result[:mfccs.shape[0],:mfccs.shape[1]] = mfccs
    s_arr[i] = result

    i += 1
    
  return s_arr

In [ ]:
s_arr = extractMFCCs(tess_AudioData)

In [ ]:
np.save('gdrive/My Drive/FYP Code/Save Data/tess_mfccs.npy',s_arr)

In [ ]:
s_arr = extractMFCCs(tess_AudioDataWN)

In [ ]:
np.save('gdrive/My Drive/FYP Code/Save Data/tess_mfccsWN.npy',s_arr)

In [ ]:
s_arr = extractMFCCs(tess_AudioDataMA)

In [ ]:
np.save('gdrive/My Drive/FYP Code/Save Data/tess_mfccsMA.npy',s_arr)

# Merging Data

In [ ]:
def mergeData(dataset1,dataset2,dataType):
  Data1 = np.load('gdrive/My Drive/FYP Code/Save Data/' + dataset1 + '_' + dataType + '.npy')
  Data2 = np.load('gdrive/My Drive/FYP Code/Save Data/' + dataset2 + '_' + dataType + '.npy')
  output = np.append(Data1,Data2,axis=0)
  return output

In [ ]:
dataType = "mfccs"
ravdessData = np.load('gdrive/My Drive/FYP Code/Save Data/ravdess_' + dataType + '.npy')
tessData = np.load('gdrive/My Drive/FYP Code/Save Data/tess_' + dataType + '.npy')

In [ ]:
ravdessData.shape

In [ ]:
output = np.append(ravdessData,tessData,axis=0)
print(ravdessData.shape,output.shape)

In [ ]:
output[:ravdessData.shape[0]]=ravdessData

In [ ]:
output[-tessData.shape[0]:]=tessData

In [ ]:
dataTypes = ["mfccs","mfccsMA","mfccsWN","Labels","Actors"]

for x in dataTypes:
  data = mergeData("ravdess","tess",x)
  np.save('gdrive/My Drive/FYP Code/Save Data/data_' + x + '.npy', data)

In [ ]:
file=np.load('gdrive/My Drive/FYP Code/Save Data/data_mfccs.npy')
file.shape

In [ ]:
file
